<a href="https://colab.research.google.com/github/rizal476/CV2020/blob/main/AppVideo_FIX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use:

* Click This Box
* Shift + Enter all the box
* To stop the webcam capture, click red text or the picture

In [ ]:
!pip install face-alignment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import base64
import html
import io
import time
import face_alignment
from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

In [ ]:
#get all face coordinates using Face Alignment
def getFaceCoordinate(img):
  cv2_im = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #convert img color
  fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=True) #load Face Alignment
  preds = fa.get_landmarks(cv2_im)[-1] #get all face coordinates
  
  return preds #return all face coordinates

In [ ]:
#calculate euclidean distance for 8 fix points
def hitung_euclidean(koordinat):
  distance = []
  for i in range(len(koordinat)): #looping for the 8 points
    for j in range(i): 
      distance.append(np.linalg.norm(koordinat[i]-koordinat[j])) #save the euclidean distances between two points
  return np.array(distance) #return all euclidean distances

In [ ]:
import argparse
from sys import platform
import tensorflow as tf
from tensorflow.keras.models import load_model

model_prediksi = load_model('/content/drive/My Drive/FG/train 7 oktober 2020/model_lstm_rizal_10 oktober 2020 6:02pm.h5') #load model 
buffer = [] #to store frames

#conver from js object (camera) to image numpy array
def js_object_to_img_array(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam

    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes)) #open image using Image PIL
    image_array = np.array(image_PIL) #change to numpy array

    return image_array #return image (numpy array)

#classifying image
def classifying_img(image_array): 
    faceCoor = getFaceCoordinate(image_array) #get all face coordinates
    fix_index = [8, 2, 14, 30, 48, 54] #the determined 8 fix points index for face
    faceCoorFix = [faceCoor[x] for x in fix_index ] #get the 8 fix points
    faceCoorFix = hitung_euclidean(faceCoorFix) #calculate the euclidean distaces for all points
    if len(buffer) == 40: #if buffer contains 40 frames
        buffer.pop(0) #pop 1 frame

    buffer.append(faceCoorFix) #store euclidean distances for 8 points
    y_pred = -1
    if len(buffer) == 40: #if buffer contains 40 frames
      buffer2 = np.array(buffer)
      buffer2 = np.expand_dims(buffer2,0)
      y_pred = model_prediksi.predict(buffer2) #predict using the euclidean distances
      print(np.round(y_pred, 3)) 

    return y_pred #return predict

# Try Webcam

In [ ]:
start_input()
label_html = 'Capturing...'
img_data = ''

while True:
  js_reply = take_photo(label_html, img_data)
  if not js_reply:
    break
  image = js_object_to_img_array(js_reply)
  y_pred= classifying_img(image)


<IPython.core.display.Javascript object>

[[1.2425653e-04 6.3547236e-07 3.6043103e-03 2.6952026e-03 9.9357563e-01]]
[[3.1854496e-03 9.9156946e-01 5.1368014e-03 2.4668037e-05 8.3571322e-05]]
[[4.1535329e-03 9.5581836e-01 3.9958082e-02 6.8936453e-05 9.9932060e-07]]
[[4.4973241e-03 4.4707934e-04 1.9353710e-01 8.0139071e-01 1.2777165e-04]]
[[2.6366243e-03 2.1700826e-06 6.8859110e-04 9.9612075e-01 5.5181014e-04]]
[[2.3850633e-01 2.3784185e-06 5.0654111e-04 7.4724662e-01 1.3738190e-02]]
[[7.1845627e-03 5.6148639e-07 1.0455427e-02 9.8009723e-01 2.2621895e-03]]
[[1.0438766e-02 3.5074146e-07 3.9778589e-04 9.8152268e-01 7.6403590e-03]]
[[1.6874027e-04 8.9412475e-07 3.0975381e-04 7.9153836e-01 2.0798218e-01]]
[[1.1851051e-02 9.3215766e-07 4.7683958e-03 9.3353236e-01 4.9847320e-02]]
[[6.2794625e-03 6.5331699e-07 3.2520974e-03 9.8277032e-01 7.6975157e-03]]
[[5.1875077e-02 1.7029660e-06 2.5071612e-02 9.0402901e-01 1.9022550e-02]]
[[1.8240776e-02 9.1054164e-05 4.2490275e-03 4.4294526e-03 9.7298968e-01]]
[[1.6488643e-02 9.0890309e-05 2.135716